In [50]:
import plotly.express as px
import pandas as pd
import numpy as np

In [53]:
benches = pd.read_json('results.json')
benches['text'] = benches['seconds'].apply(str)
benches.loc[benches.seconds == 600, "text"] = "Time Out"
benches.loc[benches.seconds == 0, ["seconds","text"]] = 600,"Not Implemented"
for name, group in benches.groupby(['operation']):
    fig = px.bar(group, 
        title=name,
        x='dataset', 
        y='seconds',
        color='backend',
        barmode='group',
        text="text",
        log_y=True,
    )
    fig.update_layout(
    title_x=0.5,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        x=0.3
    )    )
    fig.update_traces(textposition = "outside")
    fig.show()

       seconds                    text
0     0.007730    0.007729530300000001
1     0.012113            0.0121133327
2   600.000000         Not Implemented
3     0.016452            0.0164523125
4     0.255008            0.2550075054
5     0.024072            0.0240716934
6     0.000027             2.67029e-05
7   600.000000                Time Out
8     0.000036  3.6239600000000005e-05
9   600.000000                Time Out
10  600.000000         Not Implemented
11    0.002413             0.002412796
12  600.000000                Time Out
13  600.000000         Not Implemented
14  600.000000                Time Out
15    0.010140    0.010140419000000001
16    0.010463            0.0104632378
17  600.000000         Not Implemented
18    0.013575            0.0135746002
19    0.212609            0.2126085758
20    0.162495            0.1624953747
21    0.000031              3.0756e-05
22  600.000000                Time Out
23    0.000039             3.88622e-05
24  600.000000           